# Predict (Forecast)

### Setup

In [32]:
# import pandas
import pandas as pd

# import matplotlib
%matplotlib inline
import matplotlib

plt = matplotlib.pyplot
matplotlib.rcParams['figure.figsize'] = [15, 8]

# import requests
import requests

# import keras helpers
from keras.models import load_model

# misc
import os
from dotenv import load_dotenv
load_dotenv()
import pickle

### Retrieve 3-day Forecast

In [30]:
res = requests.get('https://api.weatherapi.com/v1/forecast.json?key={}&q=Callicoon, NY&days=3&aqi=no&alerts=no'.format(os.environ['WEATHER_KEY']))

forecast_raw = []
for day in res.json()['forecast']['forecastday']:
  forecast_raw += day['hour']

forecast = {}
for hour in forecast_raw:
  airtemp = hour['temp_c']
  cloudcover = hour['cloud'] * (4 / 100)
  precip = 0
  if (hour['will_it_rain']): precip = 2
  elif (hour['will_it_snow']): precip = 3

  forecast[pd.to_datetime(hour['time'])] = {
    'airtemp': airtemp,
    'cloudcover': cloudcover,
    'precip': precip
  }

forecast_df = pd.DataFrame.from_dict(forecast, orient='index')
forecast_df = forecast_df.resample('15min').interpolate('linear')

print(forecast_df.head(20))

                     airtemp  cloudcover  precip
2022-06-23 00:00:00   14.300         4.0     0.0
2022-06-23 00:15:00   14.225         4.0     0.5
2022-06-23 00:30:00   14.150         4.0     1.0
2022-06-23 00:45:00   14.075         4.0     1.5
2022-06-23 01:00:00   14.000         4.0     2.0
2022-06-23 01:15:00   13.950         4.0     2.0
2022-06-23 01:30:00   13.900         4.0     2.0
2022-06-23 01:45:00   13.850         4.0     2.0
2022-06-23 02:00:00   13.800         4.0     2.0
2022-06-23 02:15:00   13.775         4.0     2.0
2022-06-23 02:30:00   13.750         4.0     2.0
2022-06-23 02:45:00   13.725         4.0     2.0
2022-06-23 03:00:00   13.700         4.0     2.0
2022-06-23 03:15:00   13.700         4.0     2.0
2022-06-23 03:30:00   13.700         4.0     2.0
2022-06-23 03:45:00   13.700         4.0     2.0
2022-06-23 04:00:00   13.700         4.0     2.0
2022-06-23 04:15:00   13.800         4.0     2.0
2022-06-23 04:30:00   13.900         4.0     2.0
2022-06-23 04:45:00 

### Retrieve Model and Scaler

In [45]:
model = load_model('output/model.h5')
in_scaler = pickle.load(open('output/in_scaler.pickle', 'rb'))
out_scaler = pickle.load(open('output/out_scaler.pickle', 'rb'))

### Scale Forecast Data

In [59]:
values = forecast_df.values
values = values.astype('float')

scaled = in_scaler.transform(values)

input = pd.DataFrame(scaled, columns=['var1(t-1)', 'var2(t-1)', 'var3(t-1)'])
test_X = input.values.reshape((input.shape[0], 1, input.shape[1]))
print(test_X)

[[[0.49753086 0.8        0.        ]]

 [[0.49660494 0.8        0.16666667]]

 [[0.49567901 0.8        0.33333333]]

 [[0.49475309 0.8        0.5       ]]

 [[0.49382716 0.8        0.66666667]]

 [[0.49320988 0.8        0.66666667]]

 [[0.49259259 0.8        0.66666667]]

 [[0.49197531 0.8        0.66666667]]

 [[0.49135802 0.8        0.66666667]]

 [[0.49104938 0.8        0.66666667]]

 [[0.49074074 0.8        0.66666667]]

 [[0.4904321  0.8        0.66666667]]

 [[0.49012346 0.8        0.66666667]]

 [[0.49012346 0.8        0.66666667]]

 [[0.49012346 0.8        0.66666667]]

 [[0.49012346 0.8        0.66666667]]

 [[0.49012346 0.8        0.66666667]]

 [[0.49135802 0.8        0.66666667]]

 [[0.49259259 0.8        0.66666667]]

 [[0.49382716 0.8        0.66666667]]

 [[0.49506173 0.8        0.66666667]]

 [[0.49475309 0.8        0.66666667]]

 [[0.49444444 0.8        0.66666667]]

 [[0.4941358  0.8        0.66666667]]

 [[0.49382716 0.8        0.66666667]]

 [[0.49290123 0.8        

### Run Prediction

In [61]:
yhat = model.predict(test_X)
print(yhat)

ValueError: in user code:

    File "/opt/homebrew/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/homebrew/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/opt/homebrew/anaconda3/envs/tf/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/homebrew/anaconda3/envs/tf/lib/python3.10/site-packages/keras/e